**Install OpenMP support for C++ by running the following command:**

In [ ]:
!apt-get install g++-8
!ln -s /usr/bin/g++-8 /usr/local/bin/g++


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cpp-8 gcc-8 gcc-8-base libgcc-8-dev libmpx2 libstdc++-8-dev
Suggested packages:
  gcc-8-locales g++-8-multilib gcc-8-doc gcc-8-multilib libstdc++-8-doc
The following NEW packages will be installed:
  cpp-8 g++-8 gcc-8 gcc-8-base libgcc-8-dev libmpx2 libstdc++-8-dev
0 upgraded, 7 newly installed, 0 to remove and 23 not upgraded.
Need to get 32.8 MB of archives.
After this operation, 115 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 gcc-8-base amd64 8.4.0-3ubuntu2 [18.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 cpp-8 amd64 8.4.0-3ubuntu2 [8,945 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libmpx2 amd64 8.4.0-3ubuntu2 [11.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libgcc-8-dev amd64 8.4.0-3ubuntu2 [2,313 kB]
Get:5 http://a

**C++ program code (including any required header files)**

In [ ]:
code="""
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define MAX_NODES 10000

int n;
int adj_list[MAX_NODES][MAX_NODES];
int visited[MAX_NODES];
int global_queue[MAX_NODES];
int queue_size;

void parallel_bfs(int start) {
    visited[start] = 1;
    global_queue[0] = start;
    queue_size = 1;

    while (queue_size > 0) {
        int num_threads = omp_get_max_threads();
        int chunk_size = (queue_size + num_threads - 1) / num_threads;
        int local_queue[MAX_NODES];
        int local_queue_size = 0;

        #pragma omp parallel for shared(local_queue, local_queue_size)
        for (int i = 0; i < queue_size; i++) {
            int node = global_queue[i];

            for (int j = 0; j < n; j++) {
                if (adj_list[node][j] && !visited[j]) {
                    visited[j] = 1;
                    local_queue[local_queue_size++] = j;
                }
            }
        }

        queue_size = 0;

        #pragma omp parallel for
        for (int i = 0; i < num_threads; i++) {
            int start = i * chunk_size;
            int end = start + chunk_size;
            if (end > local_queue_size) {
                end = local_queue_size;
            }

            for (int j = start; j < end; j++) {
                global_queue[queue_size++] = local_queue[j];
            }
        }
    }
}

int main() {
    int start_node;

    printf("Enter the number of nodes: ");
    scanf("%d", &n);

    printf("Enter the adjacency matrix: \n");
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            scanf("%d", &adj_list[i][j]);
        }
    }

    printf("Enter the starting node: ");
    scanf("%d", &start_node);

    parallel_bfs(start_node);

    printf("The BFS traversal order is: ");
    for (int i = 0; i < n; i++) {
        if (visited[i]) {
            printf("%d ", i);
        }
    }
    printf("\n");

    return 0;
}

"""

**Compile your C++ program with OpenMP support**

In [ ]:
#!g++-8 -fopenmp -o code code.cpp

In [ ]:
!g++-8 -fopenmp -o code code.c

**Create an input file with the necessary input data for your program and save it to the notebook's file system**

In [ ]:
%%writefile input_file.txt
5 6
0 1
0 2
1 3
1 4
2 4
3 4

Writing input_file.txt


**Run your program with input**

In [ ]:
!./code < input_file.txt

In [ ]:
!echo "5 6\n0 1\n0 2\n1 3\n1 4\n2 4\n3 4" | ./code